In [3]:
import sys
sys.path.append('/home/ec2-user/SageMaker/mariano/repositories/tdmstudio-high-recall-information-retrieval-system/')
from utils.newsgroup20.dataset import Dataset20NG, DataItem20NG
representations = Dataset20NG.get_20newsgroup_representations(type_='glove')

representations file found, loading pickle (/home/ec2-user/SageMaker/mariano/datasets/20news-18828/representations/20NG_representations_glove.pickle) ... 


In [4]:
from utils.newsgroup20.models import SVM20NG


In [5]:
labeled = Dataset20NG.get_20newsgroup_labeled_collection(category='alt.atheism')
model =SVM20NG()
model.fit(labeled, representations)

In [6]:
model.predict(labeled, representations)

array([1.08717388e-02, 3.31786031e-03, 1.74740624e-03, ...,
       1.58388842e-03, 8.62297580e-02, 8.37797413e-05])

In [5]:
representations['rec.motorcycles/104702'].shape

(300,)

In [6]:
import sys 
import os
import pandas as pd
sys.path.append('/home/ec2-user/SageMaker/mariano/repositories/tdmstudio-high-recall-information-retrieval-system/')
from utils.newsgroup20.dataset import Dataset20NG, DataItem20NG


from utils.newsgroup20.scal import SCAL20NG
import numpy as np
category='alt.atheism'
representations = Dataset20NG.get_20newsgroup_representations()
unlabeled = Dataset20NG.get_20newsgroup_unlabeled_collection()
oracle = Dataset20NG.get_20newsgroup_oracle(category=category)

relevants = [item for item in unlabeled if oracle[item.id_]==DataItem20NG.RELEVANT_LABEL]

labeled = list(np.random.choice(relevants, size=1))
for item in labeled:
    item.set_relevant()

labeled_ids = {item.id_ for item in labeled}
unlabeled = [item for item in unlabeled if not item.id_ in labeled_ids]


print(f'len(labeled)=  {len(set([item.id_ for item in labeled])):6,}')
print(f'len(unlabeled)={len(set([item.id_ for item in unlabeled])):6,}')


results_file = '/home/ec2-user/SageMaker/mariano/datasets/20news-18828/simulation_results/all_results.csv'


results = SCAL20NG(session_name='some session',
                   labeled_collection=labeled,
                   unlabeled_collection=unlabeled,
                   batch_size_cap=10,
                   random_sample_size=1000,
                   target_recall=0.8,
                   ranking_function='relevance',
                   item_representation=representations,
                   oracle=oracle,
                   seed=2022).run()

new_results=pd.DataFrame(results)
new_results['category']=category

if os.path.isfile(results_file):
    old_results = pd.read_csv(results_file)
    new_results = old_results.append(new_results)


new_results.to_csv(results_file, index=False)

new_results

Using BOW representation
representations file found, loading pickle ... 
len(labeled)=       1
len(unlabeled)=18,827
j= 1 - B=    2 - b= 1 - len(labeled)=     2 - len(unlabeled)=   999 - precision=1.000 - Rhat=  1.00 - tj=0.0138
j= 2 - B=    3 - b= 2 - len(labeled)=     4 - len(unlabeled)=   997 - precision=0.000 - Rhat=  1.00 - tj=0.0305
j= 3 - B=    4 - b= 3 - len(labeled)=     7 - len(unlabeled)=   994 - precision=0.667 - Rhat=  3.00 - tj=0.0252
j= 4 - B=    5 - b= 4 - len(labeled)=    11 - len(unlabeled)=   990 - precision=0.250 - Rhat=  4.00 - tj=0.0512
j= 5 - B=    6 - b= 5 - len(labeled)=    16 - len(unlabeled)=   985 - precision=0.400 - Rhat=  6.00 - tj=0.0595
j= 6 - B=    7 - b= 6 - len(labeled)=    22 - len(unlabeled)=   979 - precision=0.000 - Rhat=  6.00 - tj=0.0746
j= 7 - B=    8 - b= 7 - len(labeled)=    29 - len(unlabeled)=   972 - precision=0.286 - Rhat=  8.00 - tj=00.066
j= 8 - B=    9 - b= 8 - len(labeled)=    37 - len(unlabeled)=   964 - precision=0.500 - Rhat= 12.00

,Date,Seed,Representation,Ranking Function,Dataset,N,n,Effort,Accuracy,Precision,Recall,F1-Score,Relevant labeled,TP,TN,FP,FN,Prevalence,category
0,2022-11-17 08:53,2022,TF-IDF,relevance,20newsgroup,1000,10,275,0.956662,0.482026,0.776316,0.594758,39,590,17158,634,170,0.04557,alt.atheism
0,2022-11-17 08:53,2022,TF-IDF,relevance,20newsgroup,1000,10,275,0.959681,0.506228,0.746719,0.603393,37,569,17235,555,193,0.04494,alt.atheism


In [2]:
!rm /home/ec2-user/SageMaker/mariano/datasets/20news-18828/simulation_results/all_results.csv

In [3]:
len(unlabeled)

18827

In [4]:
len(labeled)

1

In [8]:
labeled[0].id_

'rec.motorcycles/105226'

In [9]:
oracle['rec.motorcycles/105226']

'R'

In [10]:
representations['rec.motorcycles/105226']

<1x10000 sparse matrix of type '<class 'numpy.float64'>'
	with 38 stored elements in Compressed Sparse Row format>

In [11]:
labeled[0].get_text()

'From: davet@interceptor.cds.tek.com (Dave Tharp CDS)\nSubject: Re: Looking for MOVIES w/ BIKES\n\nIn article <C5w18w.DpD@dcs.qmw.ac.uk> davide@dcs.qmw.ac.uk (Dave Edmondson) writes:\n>\n>"Eat the peach", two guys build a wall of death to alleviate boredom, get \n>mixed up with the IRA in the process.\n\n   An excellent movie.  Not essentially about motorcycles, but\na very poignant look at life in rural Ireland.\n\n-----------------------------------------------------------------------------\n| Dave Tharp                      | DoD #0751   | "You can\'t wear out       |\n| davet@interceptor.CDS.TEK.COM   | MRA #151    |   an Indian Scout,        |\n| \'88 K75S  \'48 Indian Chief      | AHRMA #751  |  Or its brother the Chief.|\n| \'75 R90S(#151) \'72 TR-2B(#751)  | AMA #524737 |  They\'re built like rocks |\n| \'65 R50/2/Velorex  \'57 NSU Max  |             |   to take the knocks,     |\n|       1936 BMW R12              | (Compulsive | It\'s the Harleys that     |\n| My employer has 

In [4]:
import pickle
vectorizer = pickle.load(open('/home/ec2-user/SageMaker/mariano/datasets/20news-18828/representations/20NG_vectorizer_bow.pickle', 'rb'))

In [12]:
vectorizer.get_feature_names_out()[representations['rec.motorcycles/105226'].toarray()[0,:]!=0]

array(['ama', 'article', 'article dave', 'bike', 'bmw', 'brother',
       'build', 'cds', 'chief', 'dave', 'death', 'dod', 'eat', 'employer',
       'essentially', 'excellent', 'grief', 'guy', 'idea', 'indian',
       'ira', 'ireland', 'knock', 'life', 'like', 'look', 'max', 'mix',
       'motorcycle', 'movie', 'process', 'rock', 'scout', 'subject',
       'subject look', 'wall', 'wear', 'write'], dtype=object)

In [11]:
(set([key for key in oracle]))==set([item.id_ for item in unlabeled]).union({labeled[0].id_})

True

In [12]:
(set([key for key in representations]))==set([item.id_ for item in unlabeled]).union({labeled[0].id_})

True

In [7]:
len(representations)

18828

In [4]:
!ls /home/ec2-user/SageMaker/mariano/datasets/20news-18828/files/rec.motorcycles | wc -l

994


In [1]:
import sys 
sys.path.append('/home/ec2-user/SageMaker/mariano/repositories/tdmstudio-high-recall-information-retrieval-system/')
from utils.newsgroup20.dataset import Dataset20NG
m = Dataset20NG.get_X()
m

Computing vectorizer
/home/ec2-user/SageMaker/mariano/datasets/20news-18828/representations/20NG_vectorizer_bow.pickle
Loading vectorizer


<18828x10000 sparse matrix of type '<class 'numpy.float64'>'
	with 1368547 stored elements in Compressed Sparse Row format>

In [5]:
import sys 
sys.path.append('/home/ec2-user/SageMaker/mariano/repositories/tdmstudio-high-recall-information-retrieval-system/')
from utils.newsgroup20.dataset import Dataset20NG
Dataset20NG.get_data()

,text,label,id
0,From: egreen@east.sun.com (Ed Green - Pixel Cr...,rec.motorcycles,104702
1,From: vech@Ra.MsState.Edu (Craig A. Vechorik)\...,rec.motorcycles,104863
2,From: asphaug@lpl.arizona.edu (Erik Asphaug x2...,rec.motorcycles,105203
3,From: nelson@seahunt.imat.com (Michael Nelson)...,rec.motorcycles,104694
4,From: jjb@dtc.hp.com (Jim Brewer)\nSubject: Re...,rec.motorcycles,105132
...,...,...,...
18823,From: mharring@cch.coventry.ac.uk (MARTIN)\nSu...,comp.graphics,38943
18824,From: capelli@vnet.IBM.COM (Ron Capelli)\nSubj...,comp.graphics,38516
18825,From: spl@ivem.ucsd.edu (Steve Lamont)\nSubjec...,comp.graphics,38292
18826,From: lewism@aix.rpi.edu (Michael C. Lewis)\nS...,comp.graphics,38219


In [7]:
v = representations['rec.motorcycles/104702']

In [9]:
from scipy import sparse
sparse.vstack([v,v])

<2x10000 sparse matrix of type '<class 'numpy.float64'>'
	with 96 stored elements in Compressed Sparse Row format>

In [7]:
import pickle
vectorizer = pickle.load(open('/home/ec2-user/SageMaker/mariano/datasets/20news-18828/representations/20NG_vectorizer_bow.pickle', 'rb'))

In [11]:
df = Dataset20NG.get_data()
df[df['id']=='104702'].iloc[0,0]

'From: egreen@east.sun.com (Ed Green - Pixel Cruncher)\nSubject: Re: Good Reasons to Wave at each other\n\nIn article 735508147@beryllium, mdennie@xerox.com (Matt Dennie) writes:\n>\n>Is it a genetic thing with little boys or what?  Even three and four\n>year old boys seem magnetically attracted to motorcycles of virtaully\n>any sort (often to the dismay of thier mothers :-)\n\nI think you just answered your own question...\n\n---\nEd Green, former Ninjaite |I was drinking last night with a biker,\n  Ed.Green@East.Sun.COM   |and I showed him a picture of you.  I said,\nDoD #0111  (919)460-8302  |"Go on, get to know her, you\'ll like her!"\n (The Grateful Dead) -->  |It seemed like the least I could do...\n\n'

In [26]:
import numpy as np
# vectorizer.transform([df[df['id']=='104702'].iloc[0,0]])
vectorizer.get_feature_names_out()[representations['104702/rec.motorcycles'].toarray()[0,:]!=0]

array(['answer', 'answer question', 'article', 'attract', 'biker', 'boy',
       'com', 'cruncher', 'dead', 'dod', 'drink', 'ed', 'ed green',
       'ed green pixel', 'genetic', 'good', 'good reason', 'grateful',
       'grateful dead', 'green', 'green pixel', 'green pixel cruncher',
       'know', 'know like', 'like', 'little', 'matt', 'mother',
       'motorcycle', 'night', 'old', 'picture', 'pixel', 'pixel cruncher',
       'question', 'reason', 'sort', 'subject', 'subject good', 'sun',
       'sun com', 'thi', 'thing', 'think', 'wave', 'write', 'year',
       'year old'], dtype=object)

In [2]:
for item in collection[:5]+collection[10000:10005]+collection[-5:]:
    print(item)

<id=104702, label=R, category=rec.motorcycles>
<id=104863, label=R, category=rec.motorcycles>
<id=105203, label=R, category=rec.motorcycles>
<id=104694, label=R, category=rec.motorcycles>
<id=105132, label=R, category=rec.motorcycles>
<id=84552, label=I, category=talk.religion.misc>
<id=83689, label=I, category=talk.religion.misc>
<id=84235, label=I, category=talk.religion.misc>
<id=84354, label=I, category=talk.religion.misc>
<id=84380, label=I, category=talk.religion.misc>
<id=38943, label=I, category=comp.graphics>
<id=38516, label=I, category=comp.graphics>
<id=38292, label=I, category=comp.graphics>
<id=38219, label=I, category=comp.graphics>
<id=39634, label=I, category=comp.graphics>


In [3]:
collection[-1].get_htmldocview()

"From: powlesla@acs.ucalgary.ca (Jim Powlesland)\nSubject: Re: PICT, EPSF, etc map of Italy\n\nIn article <93132.025641CHUNTER@UMAB.BITNET> <CHUNTER@UMAB.BITNET> writes:\n>Does anybody know where I can get a graphic (Mac PICT, EPSF, TIFF, GIF,\n>whatever) of Italy?  I'm looking for a picture of a map of Italy (even just the\n\nA map of Italy showing the states/provinces(?) is in the FreeHand\n3.1 for Windows clip art collection. Corel Draw 3.0 clip art has\nan outline map of Italy.\n\n\n\n\n-- \n/  Jim Powlesland                   / INTERNET: powlesla@acs.ucalgary.ca\n/  Academic Computing Services      / VOICE:    (403)220-7937\n/  University of Calgary            / MESSAGE:  (403)220-6201\n/  Calgary, Alberta CANADA  T2N 1N4 / FAX:      (403)282-9199\n"

In [3]:
DataItem20NG(id_='49960', label='alt.atheism').get_text()

'From: mathew <mathew@mantis.co.uk>\nSubject: Alt.Atheism FAQ: Atheist Resources\n\nArchive-name: atheism/resources\nAlt-atheism-archive-name: resources\nLast-modified: 11 December 1992\nVersion: 1.0\n\n                              Atheist Resources\n\n                      Addresses of Atheist Organizations\n\n                                     USA\n\nFREEDOM FROM RELIGION FOUNDATION\n\nDarwin fish bumper stickers and assorted other atheist paraphernalia are\navailable from the Freedom From Religion Foundation in the US.\n\nWrite to:  FFRF, P.O. Box 750, Madison, WI 53701.\nTelephone: (608) 256-8900\n\nEVOLUTION DESIGNS\n\nEvolution Designs sell the "Darwin fish".  It\'s a fish symbol, like the ones\nChristians stick on their cars, but with feet and the word "Darwin" written\ninside.  The deluxe moulded 3D plastic fish is $4.95 postpaid in the US.\n\nWrite to:  Evolution Designs, 7119 Laurel Canyon #4, North Hollywood,\n           CA 91605.\n\nPeople in the San Francisco Bay area c

In [7]:
!find /home/ec2-user/SageMaker/mariano/datasets/20news-18828/ -name 104702

/home/ec2-user/SageMaker/mariano/datasets/20news-18828/rec.motorcycles/104702
/home/ec2-user/SageMaker/mariano/datasets/20news-18828/rec.sport.baseball/104702


In [3]:
!ls /home/ec2-user/SageMaker/mariano/datasets/20news-18828/alt.atheism

49960  51196  51274  53090  53186  53285  53380  53472	53573  53759  54187
51060  51197  51275  53093  53187  53286  53381  53473	53574  53760  54188
51119  51198  51276  53094  53188  53287  53382  53474	53575  53761  54189
51120  51199  51277  53095  53190  53288  53383  53475	53579  53762  54190
51121  51200  51278  53096  53191  53289  53387  53476	53581  53763  54191
51122  51201  51279  53097  53192  53290  53389  53477	53582  53764  54192
51123  51202  51280  53098  53193  53292  53390  53478	53583  53765  54193
51124  51203  51281  53099  53194  53293  53391  53479	53586  53766  54194
51125  51204  51282  53106  53195  53294  53393  53482	53588  53780  54195
51126  51205  51283  53108  53196  53297  53394  53483	53589  53781  54196
51127  51206  51284  53110  53197  53298  53396  53486	53591  53782  54197
51128  51208  51285  53111  53198  53302  53397  53487	53592  53783  54198
51130  51209  51286  53112  53199  53303  53398  53488	53594  53784  54199
51131  51210  51287  5311

In [2]:



df = pd.DataFrame(data)
df.head()

,text,label
0,From: egreen@east.sun.com (Ed Green - Pixel Cr...,rec.motorcycles
1,From: vech@Ra.MsState.Edu (Craig A. Vechorik)\...,rec.motorcycles
2,From: asphaug@lpl.arizona.edu (Erik Asphaug x2...,rec.motorcycles
3,From: nelson@seahunt.imat.com (Michael Nelson)...,rec.motorcycles
4,From: jjb@dtc.hp.com (Jim Brewer)\nSubject: Re...,rec.motorcycles


In [3]:
%%time
X = vectorizer.fit_transform(df['text'])


CPU times: user 7min 30s, sys: 6.69 s, total: 7min 37s
Wall time: 7min 37s


In [11]:
class DataItem20NG(object):
    RELEVANT_LABEL='R'
    IRRELEVANT_LABEL='I'
    UNKNOWN_LABEL='U'
    
    def __init__(self, text, label=None):
        assert label==DataItem20NG.RELEVANT_LABEL or \
                                label==DataItem20NG.IRRELEVANT_LABEL or label==DataItem20NG.UNKNOWN_LABEL, 'Invalid label'
        self.text=text
        self.label=label
    def
DataItem20NG('some text', 'F')

AssertionError: 

In [ ]:
target_recall = 